![MLU Logo](https://drive.corp.amazon.com/view/bwernes@/MLU_Logo.png?download=true)

## Lecture 2 Support Notebook
# TOP

### Table of Contents
<p>
<div class="lev1">
    <a href="#Law-of-Large-Numbers">
        <span class="toc-item-num">1&nbsp;&nbsp;</span>
        Law of Large Numbers
    </a>
</div>
<div class="lev1">
    <a href="#Cleaning-Robot-GridWorld"><span class="toc-item-num">2.&nbsp;&nbsp;</span>
        Cleaning Robot GridWorld
    </a>
</div>
<div class="lev1">
    <a href="#Monte-Carlo-Method"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>
        Monte Carlo Method
    </a>
</div>
<div class="lev1">
    <a href="#MC-Exploring-Starts"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>
        MC Exploring Starts
    </a>
</div>
<div class="lev1">
    <a href="#MC-Control"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>
        MC Control
    </a>
</div>
<div class="lev1">
    <a href="#TD(0)"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>
        TD(0)
    </a>
</div>
<div class="lev1">
    <a href="#TD(lambda)"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>
        TD(lambda)
    </a>
</div>
<div class="lev1">
    <a href="#SARSA"><span class="toc-item-num">5&nbsp;&nbsp;</span>
        SARSA
    </a>
</div>
<div class="lev1">
    <a href="#Q-learning"><span class="toc-item-num">5&nbsp;&nbsp;</span>
        Q-learning
    </a>
</div>

In [1]:
#!/usr/bin/env python

#MIT License
#Copyright (c) 2017 Massimiliano Patacchiola
#
#Permission is hereby granted, free of charge, to any person obtaining a copy
#of this software and associated documentation files (the "Software"), to deal
#in the Software without restriction, including without limitation the rights
#to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
#copies of the Software, and to permit persons to whom the Software is
#furnished to do so, subject to the following conditions:
#
#The above copyright notice and this permission notice shall be included in all
#copies or substantial portions of the Software.
#
#THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
#IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
#FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
#AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
#LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
#OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
#SOFTWARE.

In [2]:
# INSTALL THIS PACKAGE THE FIRST TIME YOU RUN THIS INSTANCE
# !pip install tqdm

In [1]:
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
#%pylab inline
import random
%matplotlib inline

import numpy as np
import sys

if "../" not in sys.path:
  sys.path.append("../") # this line is for use in regular SageMaker
if "MLU-Repo-RL/RL/" not in sys.path:
    sys.path.append("MLU-Repo-RL/RL/") # this line is for use in SageMaker Studio
from lib_rl.gridworld import GridWorld

# Law of Large Numbers
Rolling a six-sided dice produces the expectation
(1+2+3+4+5+6)/6=3.5


In [3]:
# Trowing a dice for N times and evaluating the expectation
dice = np.random.randint(low=1, high=7, size=3)
print("Expectation (rolling 3 times): " + str(np.mean(dice)))
dice = np.random.randint(low=1, high=7, size=10)
print("Expectation (rolling 10 times): " + str(np.mean(dice)))
dice = np.random.randint(low=1, high=7, size=100)
print("Expectation (rolling 100 times): " + str(np.mean(dice)))
dice = np.random.randint(low=1, high=7, size=1000)
print("Expectation (rolling 1000 times): " + str(np.mean(dice)))
dice = np.random.randint(low=1, high=7, size=100000)
print("Expectation (rolling 100000 times): " + str(np.mean(dice)))

Expectation (rolling 3 times): 2.3333333333333335
Expectation (rolling 10 times): 3.1
Expectation (rolling 100 times): 3.51
Expectation (rolling 1000 times): 3.534
Expectation (rolling 100000 times): 3.49887


## Learn
As you can see the estimation of the expectation converges to the true value of 3.5. <br/>
What we are doing in MC reinforcement learning is exactly the same but in this case we want to estimate the **value** for each state based on the return of each episode. <br/>
As for the dice, more episodes we take into account more accurate our estimation will be.

<a href="#TOP">
        <span class="toc-item-num">&nbsp;&nbsp;</span>
        TOP
</a>

# Cleaning Robot GridWorld
+ The class called GridWorld creates a grid world of any size, add obstacles and terminal states. 
+ The cleaning robot will move in the grid world following a specific policy. Let’s bring to life our 4x3 world.
The class GridWorld has many similarities with [OpenAI Gym package](https://gym.openai.com/):
+ The method **step()** moves forward at t+1 and returns:
    + the **reward**, 
    + the **observation** (position of the robot), and 
    + a variable called **done** which is True when the episode is finished (the robot reached a terminal state).


In [4]:
# Declare our environmnet variable
# The world has 3 rows and 4 columns
env = GridWorld(3, 4)
# Define the state matrix
# Adding obstacle at position (1,1)
# Adding the two terminal states
state_matrix = np.zeros((3,4))
state_matrix[0, 3] = 1
state_matrix[1, 3] = 1
state_matrix[1, 1] = -1
print(state_matrix)

[[ 0.  0.  0.  1.]
 [ 0. -1.  0.  1.]
 [ 0.  0.  0.  0.]]


In [5]:
# Define the reward matrix
# The reward is -0.04 for all states but the terminal
reward_matrix = np.full((3,4), -0.04)
reward_matrix[0, 3] = 1
reward_matrix[1, 3] = -1
# Define the transition matrix
# For each one of the four actions there is a probability
transition_matrix = np.array([[0.8, 0.1, 0.0, 0.1],
                              [0.1, 0.8, 0.1, 0.0],
                              [0.0, 0.1, 0.8, 0.1],
                              [0.1, 0.0, 0.1, 0.8]])
print(transition_matrix)

[[0.8 0.1 0.  0.1]
 [0.1 0.8 0.1 0. ]
 [0.  0.1 0.8 0.1]
 [0.1 0.  0.1 0.8]]


In [6]:
# Define the policy matrix
# 0=UP, 1=RIGHT, 2=DOWN, 3=LEFT, NaN=Obstacle, -1=NoAction
# This is the optimal policy for world with reward=-0.04
policy_matrix = np.array([[1,      1,  1,  -1],
                          [0, np.NaN,  0,  -1],
                          [0,      3,  3,   3]])
# Set the matrices 
env.setStateMatrix(state_matrix)
env.setRewardMatrix(reward_matrix)
env.setTransitionMatrix(transition_matrix)
print (policy_matrix)

[[ 1.  1.  1. -1.]
 [ 0. nan  0. -1.]
 [ 0.  3.  3.  3.]]


+ In a few lines I defined a grid world with the properties of our example.
+ The policy is the optimal policy for a reward of -0.04 as we saw in the first lecture. 
+ Let's reset the environment (move the robot to starting position) and using the render() method to display the world.

In [7]:
#Reset the environment
observation = env.reset()
#Display the world printing on terminal
env.render()

 -  -  -  * 
 -  #  -  * 
 ○  -  -  - 



Now we can run an episode using a for loop:

In [8]:
for _ in range(1000):
    action = policy_matrix[observation[0], observation[1]]
    observation, reward, done = env.step(action)
    print("")
    print("ACTION: " + str(action))
    print("REWARD: " + str(reward))
    print("DONE: " + str(done))
    env.render()
    if done: break


ACTION: 0.0
REWARD: -0.04
DONE: False
 -  -  -  * 
 ○  #  -  * 
 -  -  -  - 


ACTION: 0.0
REWARD: -0.04
DONE: False
 ○  -  -  * 
 -  #  -  * 
 -  -  -  - 


ACTION: 1.0
REWARD: -0.04
DONE: False
 ○  -  -  * 
 -  #  -  * 
 -  -  -  - 


ACTION: 1.0
REWARD: -0.04
DONE: False
 -  ○  -  * 
 -  #  -  * 
 -  -  -  - 


ACTION: 1.0
REWARD: -0.04
DONE: False
 -  -  ○  * 
 -  #  -  * 
 -  -  -  - 


ACTION: 1.0
REWARD: 1.0
DONE: True
 -  -  -  ○ 
 -  #  -  * 
 -  -  -  - 



<a href="#TOP">
        <span class="toc-item-num">&nbsp;&nbsp;</span>
        TOP
</a>

# Monte Carlo Method
Here I will use: 
+ A discount factor of $\gamma=0.999$
+ The best policy $\pi^*$
+ The same transition model used in the previous lecture*. 

*Remember that with the current transition model the robot will go in the desired direction only in 80% of the cases. 
### First, The Return Function
$G(t) = R_{t+1} + \gamma R_{t+2} + ... = \sum_{t=0}^ {\infty} \gamma^t R(S_{t})$

In [2]:
def get_return(state_list, gamma):
    """"
    Summary line.
    ------------
    Function that estimates the return
    
    Parameters:
    ----------
        state_list: tuple
            List containing a tuple (position, reward).
        gamma: float
            The discount factor gamma.
        
    Returns:
    -------
        return_value: float
            A value representing the return for that action list. 
    """
    
    counter = 0
    return_value = 0
    for visit in state_list:
        reward = visit[1]
        return_value += reward * np.power(gamma, counter)
        counter += 1
    return return_value

In [9]:
# Defining an empty utility matrix
utility_matrix = np.zeros((3,4))
# init with 1.0e-10 to avoid division by zero
running_mean_matrix = np.full((3,4), 1.0e-10) 
gamma = 0.999 #discount factor
tot_epoch = 50000
print_epoch = 10000

for epoch in range(tot_epoch):
    #Starting a new episode
    episode_list = list()
    #Reset and return the first observation
    observation= env.reset(exploring_starts=False)
    for _ in range(1000):
        # Take the action from the action matrix
        action = policy_matrix[observation[0], observation[1]]
        # Move one step in the environment and get obs and reward
        observation, reward, done = env.step(action)
        # Append the visit in the episode list
        episode_list.append((observation, reward))
        if done: break
    # The episode is finished, now estimating the utilities
    counter = 0
    # Checkup to identify if it is the first visit to a state
    checkup_matrix = np.zeros((3,4))
    # This cycle is the implementation of First-Visit MC.
    # For each state stored in the episode list it checks if it
    # is the first visit and then estimates the return.
    for visit in episode_list:
        observation = visit[0]
        row = observation[0]
        col = observation[1]
        reward = visit[1]
        if(checkup_matrix[row, col] == 0):
            return_value = get_return(episode_list[counter:], gamma)
            running_mean_matrix[row, col] += 1
            utility_matrix[row, col] += return_value
            checkup_matrix[row, col] = 1
        counter += 1
    if(epoch % print_epoch == 0):
        print("Utility matrix after " + str(epoch+1) + " iterations:") 
        print(utility_matrix / running_mean_matrix)

#Time to check the state-value matrix obtained
print("Utility matrix after " + str(tot_epoch) + " iterations:")
print(utility_matrix / running_mean_matrix)

Utility matrix after 1 iterations:
[[0.87712296 0.918041   0.959      1.        ]
 [0.79540959 0.         0.         0.        ]
 [0.75461418 0.71385957 0.         0.        ]]
Utility matrix after 10001 iterations:
[[ 0.80913261  0.86613743  0.9169496   1.        ]
 [ 0.75830684  0.          0.65198055 -1.        ]
 [ 0.6992423   0.6370535   0.          0.        ]]
Utility matrix after 20001 iterations:
[[ 0.8086875   0.86591813  0.9168124   1.        ]
 [ 0.75777601  0.          0.6535633  -1.        ]
 [ 0.70261592  0.64585796  0.          0.        ]]
Utility matrix after 30001 iterations:
[[ 0.80673041  0.86404303  0.91496372  1.        ]
 [ 0.75581875  0.          0.64488089 -1.        ]
 [ 0.70016255  0.64756783  0.          0.        ]]
Utility matrix after 40001 iterations:
[[ 0.80598793  0.86349744  0.91446687  1.        ]
 [ 0.75498787  0.          0.64002953 -1.        ]
 [ 0.69778903  0.6464032   0.          0.        ]]
Utility matrix after 50000 iterations:
[[ 0.8057625

The script above will print the estimation of the state-value matrix every 1000 iterations.

<a href="#TOP">
        <span class="toc-item-num">&nbsp;&nbsp;</span>
        TOP
</a>

# MC Exploring Starts
+ To enable the exploring starts in our code the only thing to do is to set the parameter exploring_strarts in the reset() function to True.
+ Every time a new episode begins, the robot will start from a random position. 

In [18]:
# Defining an empty utility matrix
utility_matrix = np.zeros((3,4))
# init with 1.0e-10 to avoid division by zero
running_mean_matrix = np.full((3,4), 1.0e-10) 
gamma = 0.999 #discount factor
tot_epoch = 50000
print_epoch = 10000

for epoch in range(tot_epoch):
    #Starting a new episode
    episode_list = list()
    #Reset and return the first observation
    observation= env.reset(exploring_starts=True)
    for _ in range(1000):
        # Take the action from the action matrix
        action = policy_matrix[observation[0], observation[1]]
        # Move one step in the environment and get obs and reward
        observation, reward, done = env.step(action)
        # Append the visit in the episode list
        episode_list.append((observation, reward))
        if done: break
    # The episode is finished, now estimating the utilities
    counter = 0
    # Checkup to identify if it is the first visit to a state
    checkup_matrix = np.zeros((3,4))
    # This cycle is the implementation of First-Visit MC.
    # For each state stored in the episode list it checks if it
    # is the first visit and then estimates the return.
    for visit in episode_list:
        observation = visit[0]
        row = observation[0]
        col = observation[1]
        reward = visit[1]
        if(checkup_matrix[row, col] == 0):
            return_value = get_return(episode_list[counter:], gamma)
            running_mean_matrix[row, col] += 1
            utility_matrix[row, col] += return_value
            checkup_matrix[row, col] = 1
        counter += 1
    if(epoch % print_epoch == 0):
        print("Utility matrix after " + str(epoch+1) + " iterations:") 
        print(utility_matrix / running_mean_matrix)

#Time to check the utility matrix obtained
print("Utility matrix after " + str(tot_epoch) + " iterations:")
print(utility_matrix / running_mean_matrix)

Utility matrix after 1 iterations:
[[0.87712296 0.918041   0.959      1.        ]
 [0.75461418 0.         0.         0.        ]
 [0.71385957 0.67314571 0.63247256 0.        ]]
Utility matrix after 10001 iterations:
[[ 0.80844337  0.8664559   0.91754396  1.        ]
 [ 0.75841854  0.          0.65201451 -1.        ]
 [ 0.70068058  0.64900819  0.5966106   0.40447537]]
Utility matrix after 20001 iterations:
[[ 0.80953839  0.8669877   0.91842743  1.        ]
 [ 0.75852686  0.          0.66863796 -1.        ]
 [ 0.69977438  0.64807495  0.60166081  0.38997887]]
Utility matrix after 30001 iterations:
[[ 0.80903855  0.86577214  0.91730376  1.        ]
 [ 0.75747417  0.          0.6629768  -1.        ]
 [ 0.69846782  0.64578443  0.60272715  0.40829719]]
Utility matrix after 40001 iterations:
[[ 0.80871151  0.86544232  0.91707062  1.        ]
 [ 0.75734168  0.          0.65244006 -1.        ]
 [ 0.69947116  0.64743188  0.6017003   0.40543337]]
Utility matrix after 50000 iterations:
[[ 0.8085998

<a href="#TOP">
        <span class="toc-item-num">&nbsp;&nbsp;</span>
        TOP
</a>

# MC Control
We will use again the function **get_return()** but this time the input will be a list containing the tuple (observation, action, reward).

In [10]:
def get_return(state_list, gamma):
    '''Get the return for a list of action-state values.
    @return get the Return
    '''
    counter = 0
    return_value = 0
    for visit in state_list:
        reward = visit[2]
        return_value += reward * np.power(gamma, counter)
        counter += 1
    return return_value

We will use another new function called **update_policy()**, which will make the policy greedy with respect to the current state-action function.

In [11]:
def update_policy(episode_list, policy_matrix, state_action_matrix):
    '''Update a policy making it greedy in respect of the state-action matrix.
    @return the updated policy
    '''
    for visit in episode_list:
        observation = visit[0]
        col = observation[1] + (observation[0]*4)
        if(policy_matrix[observation[0], observation[1]] != -1):      
            policy_matrix[observation[0], observation[1]] = \
                np.argmax(state_action_matrix[:,col])
    return policy_matrix

+ The update_policy() function is part of the improvement step of the GPI and it is fundamental in order to get convergence to an optimal policy. 
+ I will use also the function print_policy() which I already used in the previous lecture in order to print on terminal the policy using the symbols: ^, >, v, <, *, #. 
+ In the main() function, I initialized a random policy matrix and the state_action_matrix that contains the utilities of each state-action pair. 
+ The matrix can be initialised to zeros or to random values, it does not matter.

In [12]:
def print_policy(policy_matrix):
    '''Print the policy using specific symbol.
    * terminal state
    ^ > v < up, right, down, left
    # obstacle
    '''
    counter = 0
    shape = policy_matrix.shape
    policy_string = ""
    for row in range(shape[0]):
        for col in range(shape[1]):
            if(policy_matrix[row,col] == -1): policy_string += " *  "            
            elif(policy_matrix[row,col] == 0): policy_string += " ^  "
            elif(policy_matrix[row,col] == 1): policy_string += " >  "
            elif(policy_matrix[row,col] == 2): policy_string += " v  "           
            elif(policy_matrix[row,col] == 3): policy_string += " <  "
            elif(np.isnan(policy_matrix[row,col])): policy_string += " #  "
            counter += 1
        policy_string += '\n'
    print(policy_string)

Here we have the main loop of the algorithm, which is not so different from the loop used for MC prediction.

In [13]:


env = GridWorld(3, 4)

#Define the state matrix
state_matrix = np.zeros((3,4))
state_matrix[0, 3] = 1
state_matrix[1, 3] = 1
state_matrix[1, 1] = -1
print("State Matrix:")
print(state_matrix)

#Define the reward matrix
reward_matrix = np.full((3,4), -0.04)
reward_matrix[0, 3] = 1
reward_matrix[1, 3] = -1
print("Reward Matrix:")
print(reward_matrix)

#Define the transition matrix
transition_matrix = np.array([[0.8, 0.1, 0.0, 0.1],
                              [0.1, 0.8, 0.1, 0.0],
                              [0.0, 0.1, 0.8, 0.1],
                              [0.1, 0.0, 0.1, 0.8]])

#Random policy
policy_matrix = np.random.randint(low=0, high=4, size=(3, 4)).astype(np.float32)
policy_matrix[1,1] = np.NaN #NaN for the obstacle at (1,1)
policy_matrix[0,3] = policy_matrix[1,3] = -1 #No action for the terminal states

#Set the matrices in the world
env.setStateMatrix(state_matrix)
env.setRewardMatrix(reward_matrix)
env.setTransitionMatrix(transition_matrix)

state_action_matrix = np.random.random_sample((4,12)) # Q
#init with 1.0e-10 to avoid division by zero
running_mean_matrix = np.full((4,12), 1.0e-10) 
gamma = 0.999
tot_epoch = 500000
print_epoch = 3000

for epoch in range(tot_epoch):
    #Starting a new episode
    episode_list = list()
    #Reset and return the first observation and reward
    observation = env.reset(exploring_starts=True)
    #action = np.random.choice(4, 1)
    #action = policy_matrix[observation[0], observation[1]]
    #episode_list.append((observation, action, reward))
    is_starting = True
    for _ in range(1000):
        #Take the action from the action matrix
        action = policy_matrix[observation[0], observation[1]]
        #If the episode just started then it is
            #necessary to choose a random action (exploring starts)
        if(is_starting): 
            action = np.random.randint(0, 4)
            is_starting = False      
        #Move one step in the environment and get obs and reward
        new_observation, reward, done = env.step(action)
        #Append the visit in the episode list
        episode_list.append((observation, action, reward))
        observation = new_observation
        if done: break
    #The episode is finished, now estimating the utilities
    counter = 0
    #Checkup to identify if it is the first visit to a state
    checkup_matrix = np.zeros((4,12))
    #This cycle is the implementation of First-Visit MC.
    #For each state stored in the episode list check if it
    #is the rist visit and then estimate the return.
    for visit in episode_list:
        observation = visit[0]
        action = visit[1]
        col = int(observation[1] + (observation[0]*4))
        row = int(action)
        if(checkup_matrix[row, col] == 0):
            return_value = get_return(episode_list[counter:], gamma)
            running_mean_matrix[row, col] += 1
            state_action_matrix[row, col] += return_value
            checkup_matrix[row, col] = 1
        counter += 1
    #Policy Update
    policy_matrix = update_policy(episode_list, 
                                  policy_matrix, 
                                  state_action_matrix/running_mean_matrix)
    #Printing
    if(epoch % print_epoch == 0):
        print("")
        print("State-Action matrix after " + str(epoch+1) + " iterations:") 
        print(state_action_matrix / running_mean_matrix)
        print("Policy matrix after " + str(epoch+1) + " iterations:") 
        print(policy_matrix)
        print_policy(policy_matrix)
#Time to check the utility matrix obtained
print("Utility matrix after " + str(tot_epoch) + " iterations:")
print(state_action_matrix / running_mean_matrix)




State Matrix:
[[ 0.  0.  0.  1.]
 [ 0. -1.  0.  1.]
 [ 0.  0.  0.  0.]]
Reward Matrix:
[[-0.04 -0.04 -0.04  1.  ]
 [-0.04 -0.04 -0.04 -1.  ]
 [-0.04 -0.04 -0.04 -0.04]]

State-Action matrix after 1 iterations:
[[ 3.69902782e+09  3.59094881e+08  5.85374356e+09  3.52144240e+09
   7.56009261e+09  4.29084454e+09  8.37712479e+09  2.52741473e+09
   5.00177949e+09  3.56739241e+09  4.22700081e+09  2.87922958e+09]
 [ 3.07415717e+09  3.29544001e+09  8.63675359e+09  2.47506144e+09
   5.82159860e+09  5.47680377e+09  6.89448216e+09  6.20601090e+09
   2.09684655e+09  1.98840175e+09 -8.55902652e-01 -2.68741984e-01]
 [ 1.92453786e+09  2.77450819e+09  7.76453821e+09  4.44879746e+09
   6.13375308e+09  5.73480301e+09  7.41230395e+09  9.94775300e+09
   2.67184748e+09  4.39723820e+09  5.68381899e+09  9.30401715e+09]
 [ 7.73351011e+09  5.37696180e+09  5.82962471e+09  4.99416544e+09
   8.50481779e+09  8.24605060e+09  1.41149391e+09  2.67570533e+09
   7.66696915e+09  4.04547416e+09  2.87870484e+09  7.22450426

If we compare the code below with the one used in MC for prediction we will notice some important differences, for example the following condition:

if(is_starting): 
    action = np.random.randint(0, 4)
    is_starting = False 

This condition assures to satisfy the exploring starts. The MC algorithm will converge to the optimal solution only if we assure the exploring starts. In MC for control it is not sufficient to select random starting states. During the iterations the algorithm will improve the policy only if all the actions have a non-zero probability to be chosen. In this sense when the episode start we have to select a random action, this must be done only for the starting state.

There is another subtle difference that we must analyse. In the code I differentiate between observation and new_observation meaning the observation at time t and observation at time t+1. What we have to store in our episode list is the observation at t, the action taken at t and the reward obtained at t+1. Remember that we are interested in the utility of taking a certain action in a certain state.

It is time to run the script and see what we obtain. Before remember that for the special 4x3 world we already know the optimal policy. If you go back to the first post you will see that we found the optimal policy in case of reward equal to -0.04 (for non terminal states), and in case of transition model with 80-10-10 percent probabilities. This optimal policy is the following:

<img src="../images/RL-lecture2-9.png" alt="Drawing" style="width: 150px;">

In the optimal policy the robot will move far away from the stairs at state (4, 2) and will reach the charging station through the longest path. Now I will show you the evolution of the policy once we run the script for MC control estimation:

## Conclusions
At the beginning the MC method is initialized with a random policy, it is not a surprise that the first policy is a complete non-sense. 
After 3000 iteration the algorithm find a sub-optimal policy. In this policy the robot moves close to the stairs in order to reach the charging station. As we said in the previous post this is risky because the robot can fall down. At iteration 78000 the algorithm finds another policy, which is always sub-optimal, but it is slightly better than the previous one. Finally at iteration 405000 the algorithm finds the optimal policy and stick to it until the end.

The MC method cannot converge to any sub-optimal policy. Looking to the GPI scheme this is obvious. If the algorithm converges to a sub-optimal policy then the utility function would eventually converge to the utility function for that policy and that in turn would cause the policy to change. Stability is reached only when both the policy and the utility function are optimal. Convergence to this optimal fixed point seems inevitable but has not yet been formally proved.

I would like to reflect for a moment on the beauty of the MC algorithm. In MC for control the method can estimate the best policy having nothing. The robot is moving in the environment trying different actions and following the consequences of those actions until the end. That’s all. The robot does not know the reward function, it does not know the transition model and it does not have any policy to follow. Nevertheless the algorithm improves until reaching the optimal strategy.


<a href="#TOP">
        <span class="toc-item-num">&nbsp;&nbsp;</span>
        TOP
</a>

# TD(0)
Using our cleaning robot we can easily see what is the difference between TD and MC learning and what each one does at each step,

+ The update rule found in the previous part is the simplest form of TD learning, the TD(0) algorithm. 
+ TD(0) allows estimating the utility values following a specific policy. 
+ We are in the **passive learning** case for prediction, and we are in **model-free reinforcement learning**, meaning that we do not have the **transition model**. 
+ To estimate the state-value function, we can only move in the world. 
+ Let's see what does it mean to apply the TD algorithm to a single episode. 
+ I am going to use the same episode where the robot starts at (1,1) and reaches the terminal state at (4,3) after seven steps.
<img src="../images/RL-lecture2-1.png" alt="Drawing" style="width: 500px;">


Applying the TD algorithm means to move step by step considering only the state at t and the state at t+1. That’s it, after each step we get the utility value and the reward at t+1 and we update the value at t. The TD(0) algorithm ignores the past states and this is shown by the shadow I added above those states. Applying the algorithm to the episode ($\gamma=0.9, \alpha=0.1$) leads to the following changes in the utility matrix:
<img src="../images/RL-lecture2-2.png" alt="Drawing" style="width: 500px;">

+ The red frame highlights the utility value that has been updated at each visit. The matrix is initialised with zeros. 
+ At k=1 the state (1,1) is updated since the robot is in the state (1,2) and the first reward (-0.04) is available. 
+ The calculation for updating the utility at (1,1) is: 0.0 + 0.1 (-0.04 + 0.9 (0.0) - 0.0) = -0.004. Similarly to (1,1) the algorithm updates the state at (1,2). 
+ At k=3 the robot goes back and the calculation take the form: 0.0 + 0.1 (-0.04 + 0.9 (-0.004) - 0.0) = -0.00436. 
+ At k=4 the robot changes again its direction. In this case the algorithm update for the second time the state (1,2) as follow: -0.004 + 0.1 (-0.04 + 0.9 (-0.00436) + 0.004) = -0.0079924. 
+ The same process is applied until the end of the episode.

We are using the class GridWorld contained in the module gridworld.py again. <br/>
I will use again the 4x3 world with a charging station at (4,3) and the stairs at (4,2). <br/>
The optimal policy and the state-values of this world are the same we obtained in the previous example:
<img src="../images/RL-lecture2-3.png" alt="Drawing" style="width: 500px;">

The update rule of TD(0) can be implemented in a few lines:



In [17]:
def update_utility(utility_matrix, observation, new_observation, 
                   reward, alpha, gamma):
    '''Return the updated utility matrix
    @param utility_matrix the matrix before the update
    @param observation the state obsrved at t
    @param new_observation the state observed at t+1
    @param reward the reward observed after the action
    @param alpha the ste size (learning rate)
    @param gamma the discount factor
    @return the updated utility matrix
    '''
    u = utility_matrix[observation[0], observation[1]]
    u_t1 = utility_matrix[new_observation[0], new_observation[1]]
    utility_matrix[observation[0], observation[1]] += \
        alpha * (reward + gamma * u_t1 - u)
    return utility_matrix

The main loop is much simpler than the one of MC methods. In this case we do not have any first-visit constraint and the only thing to do is to apply the update rule.

In [18]:
def main():

    env = GridWorld(3, 4)

    #Define the state matrix
    state_matrix = np.zeros((3,4))
    state_matrix[0, 3] = 1
    state_matrix[1, 3] = 1
    state_matrix[1, 1] = -1
    print("State Matrix:")
    print(state_matrix)

    #Define the reward matrix
    reward_matrix = np.full((3,4), -0.04)
    reward_matrix[0, 3] = 1
    reward_matrix[1, 3] = -1
    print("Reward Matrix:")
    print(reward_matrix)

    #Define the transition matrix
    transition_matrix = np.array([[0.8, 0.1, 0.0, 0.1],
                                  [0.1, 0.8, 0.1, 0.0],
                                  [0.0, 0.1, 0.8, 0.1],
                                  [0.1, 0.0, 0.1, 0.8]])

    #Define the policy matrix
    #This is the optimal policy for world with reward=-0.04
    policy_matrix = np.array([[1,      1,  1,  -1],
                              [0, np.NaN,  0,  -1],
                              [0,      3,  3,   3]])
    print("Policy Matrix:")
    print(policy_matrix)

    env.setStateMatrix(state_matrix)
    env.setRewardMatrix(reward_matrix)
    env.setTransitionMatrix(transition_matrix)

    utility_matrix = np.zeros((3,4))
    gamma = 0.999
    alpha = 0.1 #constant step size
    tot_epoch = 300000
    print_epoch = 50000

    for epoch in range(tot_epoch):
        #Reset and return the first observation
        observation = env.reset(exploring_starts=False)
        for step in range(1000):
            #Take the action from the action matrix
            action = policy_matrix[observation[0], observation[1]]
            #Move one step in the environment and get obs and reward
            new_observation, reward, done = env.step(action)
            utility_matrix = update_utility(utility_matrix, observation, 
                                            new_observation, reward, alpha, gamma)
            observation = new_observation
            #print(utility_matrix)
            if done: break

        if(epoch % print_epoch == 0):
            print("")
            print("Utility matrix after " + str(epoch+1) + " iterations:") 
            print(utility_matrix)
    #Time to check the utility matrix obtained
    print("Utility matrix after " + str(tot_epoch) + " iterations:")
    print(utility_matrix)



if __name__ == "__main__":
    main()

State Matrix:
[[ 0.  0.  0.  1.]
 [ 0. -1.  0.  1.]
 [ 0.  0.  0.  0.]]
Reward Matrix:
[[-0.04 -0.04 -0.04  1.  ]
 [-0.04 -0.04 -0.04 -1.  ]
 [-0.04 -0.04 -0.04 -0.04]]
Policy Matrix:
[[ 1.  1.  1. -1.]
 [ 0. nan  0. -1.]
 [ 0.  3.  3.  3.]]

Utility matrix after 1 iterations:
[[-0.0076    -0.0076     0.0964     0.       ]
 [-0.004      0.        -0.0043996  0.       ]
 [-0.0076    -0.0043996  0.         0.       ]]

Utility matrix after 50001 iterations:
[[0.85630857 0.90850546 0.96124191 0.        ]
 [0.81213794 0.         0.71530317 0.        ]
 [0.74935661 0.69145512 0.         0.        ]]

Utility matrix after 100001 iterations:
[[0.84844034 0.89611715 0.93023312 0.        ]
 [0.80746268 0.         0.7503229  0.        ]
 [0.75953399 0.69124824 0.         0.        ]]

Utility matrix after 150001 iterations:
[[0.85517165 0.90061894 0.96417802 0.        ]
 [0.80291602 0.         0.72854949 0.        ]
 [0.74227987 0.67533437 0.         0.        ]]

Utility matrix after 200001 ite

<a href="#TOP">
        <span class="toc-item-num">&nbsp;&nbsp;</span>
        TOP
</a>

# TD(lambda)
The Python implementation of TD(λ) is straightforward. We only need to add an eligibility matrix and its update rule.

In [19]:
def update_utility(utility_matrix, trace_matrix, alpha, delta):
    '''Return the updated utility matrix
    @param utility_matrix the matrix before the update
    @param alpha the step size (learning rate)
    @param delta the error (Taget-OldEstimte) 
    @return the updated utility matrix
    '''
    utility_matrix += alpha * delta * trace_matrix
    return utility_matrix

In [20]:
def update_eligibility(trace_matrix, gamma, lambda_):
    '''Return the updated trace_matrix
    @param trace_matrix the eligibility traces matrix
    @param gamma discount factor
    @param lambda_ the decaying value
    @return the updated trace_matrix
    '''
    trace_matrix = trace_matrix * gamma * lambda_
    return trace_matrix

+ The main loop introduces some new components compared to the TD(0) case. 
+ We have the estimation of delta in a separate line and the management of the trace_matrix in two lines. 
+ First of all the states are increased (+1) and then they are decayed.

In [21]:
def main():

    env = GridWorld(3, 4)

    #Define the state matrix
    state_matrix = np.zeros((3,4))
    state_matrix[0, 3] = 1
    state_matrix[1, 3] = 1
    state_matrix[1, 1] = -1
    print("State Matrix:")
    print(state_matrix)

    #Define the reward matrix
    reward_matrix = np.full((3,4), -0.04)
    reward_matrix[0, 3] = 1
    reward_matrix[1, 3] = -1
    print("Reward Matrix:")
    print(reward_matrix)

    #Define the transition matrix
    transition_matrix = np.array([[0.8, 0.1, 0.0, 0.1],
                                  [0.1, 0.8, 0.1, 0.0],
                                  [0.0, 0.1, 0.8, 0.1],
                                  [0.1, 0.0, 0.1, 0.8]])

    #Define the policy matrix
    #This is the optimal policy for world with reward=-0.04
    policy_matrix = np.array([[1,      1,  1,  -1],
                              [0, np.NaN,  0,  -1],
                              [0,      3,  3,   3]])
    print("Policy Matrix:")
    print(policy_matrix)

    #Define and print the eligibility trace matrix
    trace_matrix = np.zeros((3,4))
    print("Trace Matrix:")
    print(trace_matrix)

    env.setStateMatrix(state_matrix)
    env.setRewardMatrix(reward_matrix)
    env.setTransitionMatrix(transition_matrix)

    utility_matrix = np.zeros((3,4))
    gamma = 0.999 #discount rate
    alpha = 0.1 #constant step size
    lambda_ = 0.5 #decaying factor
    tot_epoch = 300000
    print_epoch = 50000

    for epoch in range(tot_epoch):
        #Reset and return the first observation
        observation = env.reset(exploring_starts=True)        
        for step in range(1000):
            #Take the action from the action matrix
            action = policy_matrix[observation[0], observation[1]]
            #Move one step in the environment and get obs and reward
            new_observation, reward, done = env.step(action)
            #Estimate the error delta (Target - OldEstimate)
            delta = reward + gamma * utility_matrix[new_observation[0], new_observation[1]] - \
                                     utility_matrix[observation[0], observation[1]]
            #Adding +1 in the trace matrix for the state visited
            trace_matrix[observation[0], observation[1]] += 1
            #Update the utility matrix
            utility_matrix = update_utility(utility_matrix, trace_matrix, alpha, delta)
            #Update the trace matrix (decaying)
            trace_matrix = update_eligibility(trace_matrix, gamma, lambda_)
            observation = new_observation
            if done: break #return

        if(epoch % print_epoch == 0):
            print("")
            print("Utility matrix after " + str(epoch+1) + " iterations:") 
            print(utility_matrix)
            print(trace_matrix)
    #Time to check the utility matrix obtained
    print("Utility matrix after " + str(tot_epoch) + " iterations:")
    print(utility_matrix)



if __name__ == "__main__":
    main()

State Matrix:
[[ 0.  0.  0.  1.]
 [ 0. -1.  0.  1.]
 [ 0.  0.  0.  0.]]
Reward Matrix:
[[-0.04 -0.04 -0.04  1.  ]
 [-0.04 -0.04 -0.04 -1.  ]
 [-0.04 -0.04 -0.04 -0.04]]
Policy Matrix:
[[ 1.  1.  1. -1.]
 [ 0. nan  0. -1.]
 [ 0.  3.  3.  3.]]
Trace Matrix:
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

Utility matrix after 1 iterations:
[[ 0.01895203  0.04595     0.1         0.        ]
 [ 0.00546654  0.          0.          0.        ]
 [-0.00126947 -0.01034903 -0.00705441 -0.01468195]]
[[0.12462537 0.24950025 0.4995     0.        ]
 [0.06225037 0.         0.         0.        ]
 [0.03109406 0.02328946 0.00387511 0.00290246]]

Utility matrix after 50001 iterations:
[[0.82800911 0.92539816 0.98140602 0.        ]
 [0.77891536 0.         0.86450035 0.        ]
 [0.71571759 0.66310473 0.66607501 0.3330751 ]]
[[1.57655057e-01 2.53376305e-01 5.07354441e-01 0.00000000e+00]
 [7.87487006e-02 0.00000000e+00 2.28086169e-05 0.00000000e+00]
 [4.82936945e-04 3.61719889e-04 2.90696391e-08 4.37302728e

+ Comparing the final utility matrix with the one obtained without the use of eligibility traces in TD(0) you will notice similar values. 
+ What’s the advantage eligibility traces? 
    + The eligibility traces version converges faster. 

This advantage become clear when dealing with sparse reward in a large state space. In this case the eligibility trace mechanism can considerably speeds up the convergence propagating what learnt at t+1 back to the last states visited.

<a href="#TOP">
        <span class="toc-item-num">&nbsp;&nbsp;</span>
        TOP
</a>